# Section 2: New Logo Detection

This notebook addresses the challenge of detecting new logos that were not in the original training set.

## Problem:
- Customer provides an image with new logos (e.g., Adidas, NBA, Miami Heat)
- These logos may not be in the original training classes
- Need to detect and localize these new logos

## Approach:
1. **Few-shot Learning**: Use the pre-trained model as a feature extractor
2. **Template Matching**: Extract logo templates from the provided image
3. **Fine-tuning**: Fine-tune the model on new logo classes
4. **Hybrid Approach**: Combine template matching with object detection


In [ ]:
# Import necessary libraries
import os
import json
import shutil
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, ImageDraw, ImageFont
import torch
import cv2
from collections import Counter
import yaml
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Check device (MPS for Mac M1)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)


## 1. Load Section 2 Data and Analyze


In [ ]:
# Load Section 2 data
SECTION2_ROOT = Path("raw_data/logos-dataset-section-2")
section2_image = SECTION2_ROOT / "Adidas_38.jpg"
section2_label = SECTION2_ROOT / "Adidas_38.txt"

# Load and visualize the image
img = Image.open(section2_image)
print(f"Image size: {img.size}")
print(f"Image mode: {img.mode}")

# Load annotations
with open(section2_label, 'r') as f:
    annotations = f.readlines()

print(f"\nNumber of annotations: {len(annotations)}")
print("\nAnnotations:")
for ann in annotations:
    print(f"  {ann.strip()}")

# Extract new classes
new_classes = []
for ann in annotations:
    parts = ann.strip().split()
    if len(parts) >= 5:
        class_name = parts[0]
        if class_name not in new_classes:
            new_classes.append(class_name)

print(f"\nNew classes found: {new_classes}")

# Visualize the image with ground truth bounding boxes
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.imshow(img)
ax.axis('off')
ax.set_title('Section 2 Image with Ground Truth Annotations', fontsize=14, fontweight='bold')

# Draw bounding boxes
img_width, img_height = img.size
for ann in annotations:
    parts = ann.strip().split()
    if len(parts) >= 5:
        class_name = parts[0]
        x_center, y_center, width, height = [float(x) for x in parts[1:5]]
        
        # Convert normalized coordinates to pixel coordinates
        x = (x_center - width/2) * img_width
        y = (y_center - height/2) * img_height
        w = width * img_width
        h = height * img_height
        
        # Draw bounding box
        from matplotlib.patches import Rectangle
        rect = Rectangle((x, y), w, h, linewidth=3, 
                        edgecolor='red', facecolor='none')
        ax.add_patch(rect)
        ax.text(x, y-10, class_name, color='red', fontsize=12, fontweight='bold',
               bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.8))

plt.tight_layout()
plt.savefig('section2_ground_truth.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nGround truth visualization saved to section2_ground_truth.png")


## 2. Load Pre-trained Model from Section 1


In [ ]:
# Load improved model info from Section 1
try:
    with open('improved_info.json', 'r') as f:
        improved_info = json.load(f)
    print("Loaded improved model info from Section 1")
    print(f"Original classes: {len(improved_info['classes'])}")
except FileNotFoundError:
    # Fallback to baseline if improved not available
    with open('baseline_info.json', 'r') as f:
        improved_info = json.load(f)
    print("Loaded baseline model info from Section 1 (improved not found)")

original_classes = improved_info['classes']
print(f"\nOriginal classes (first 10): {original_classes[:10]}...")

# Check if new classes are in original classes
print(f"\nNew classes from Section 2: {new_classes}")
overlap = [cls for cls in new_classes if cls in original_classes]
new_only = [cls for cls in new_classes if cls not in original_classes]

print(f"\nClasses already in model: {overlap}")
print(f"Truly new classes: {new_only}")

# Install ultralytics
try:
    from ultralytics import YOLO
    print("\nUltralytics already installed")
except ImportError:
    print("Installing ultralytics...")
    import subprocess
    subprocess.check_call(["pip", "install", "ultralytics"])
    from ultralytics import YOLO

# Load the pre-trained model
pretrained_model_path = improved_info['model_path']
print(f"\nLoading pre-trained model from: {pretrained_model_path}")
pretrained_model = YOLO(pretrained_model_path)
print("Pre-trained model loaded successfully!")


## 3. Approach 1: Test Pre-trained Model on New Image


In [ ]:
# Test the pre-trained model on the new image
print("Testing pre-trained model on Section 2 image...")
predictions = pretrained_model.predict(
    source=str(section2_image),
    conf=0.25,
    iou=0.45,
    save=False,
    show=False
)

pred = predictions[0]

# Visualize predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.imshow(img)
ax.axis('off')
ax.set_title('Pre-trained Model Predictions on Section 2 Image', fontsize=14, fontweight='bold')

# Draw predictions
if pred.boxes is not None and len(pred.boxes) > 0:
    boxes = pred.boxes.xyxy.cpu().numpy()
    confidences = pred.boxes.conf.cpu().numpy()
    classes = pred.boxes.cls.cpu().numpy().astype(int)
    
    print(f"\nDetected {len(boxes)} objects:")
    for box, conf, cls in zip(boxes, confidences, classes):
        x1, y1, x2, y2 = box
        if cls < len(original_classes):
            class_name = original_classes[cls]
            print(f"  {class_name}: {conf:.3f}")
            
            # Draw bounding box
            from matplotlib.patches import Rectangle
            rect = Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2, 
                           edgecolor='lime', facecolor='none')
            ax.add_patch(rect)
            ax.text(x1, y1-5, f"{class_name} {conf:.2f}", 
                   color='lime', fontsize=10, fontweight='bold',
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='black', alpha=0.7))
else:
    print("\nNo objects detected by pre-trained model")

plt.tight_layout()
plt.savefig('section2_pretrained_predictions.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nPre-trained predictions saved to section2_pretrained_predictions.png")


## 4. Approach 2: Fine-tune Model with New Classes

Since the new logos may not be in the original training set, we'll:
1. Create a small dataset from the provided image
2. Add new classes to the model
3. Fine-tune the model on the new data


In [ ]:
# Create a dataset for fine-tuning with new classes
# We'll use the Section 2 image as training data
# In a real scenario, we'd have more images, but we'll work with what we have

SECTION2_DATASET = Path("section2_dataset")
SECTION2_DATASET.mkdir(exist_ok=True)

# Create directories
for split in ['train', 'val']:
    (SECTION2_DATASET / split / 'images').mkdir(parents=True, exist_ok=True)
    (SECTION2_DATASET / split / 'labels').mkdir(parents=True, exist_ok=True)

# Copy image to train and val (we'll use the same image for both in this case)
# In practice, you'd have more images
shutil.copy(section2_image, SECTION2_DATASET / 'train' / 'images' / section2_image.name)
shutil.copy(section2_image, SECTION2_DATASET / 'val' / 'images' / section2_image.name)

# Create YOLO format labels with new classes
# Combine original classes with new classes
all_classes_combined = original_classes + [cls for cls in new_classes if cls not in original_classes]

print(f"Total classes: {len(all_classes_combined)}")
print(f"Original: {len(original_classes)}, New: {len(new_only)}")

# Convert annotations to YOLO format
def convert_to_yolo_format(label_file, output_file, class_mapping):
    """Convert annotation file to YOLO format with class indices"""
    with open(label_file, 'r') as f:
        lines = f.readlines()
    
    with open(output_file, 'w') as f:
        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 5:
                class_name = parts[0]
                if class_name in class_mapping:
                    class_idx = class_mapping[class_name]
                    f.write(f"{class_idx} {' '.join(parts[1:5])}\n")

# Create class mapping
class_mapping = {cls: idx for idx, cls in enumerate(all_classes_combined)}

# Convert labels
convert_to_yolo_format(
    section2_label,
    SECTION2_DATASET / 'train' / 'labels' / (section2_image.stem + '.txt'),
    class_mapping
)
convert_to_yolo_format(
    section2_label,
    SECTION2_DATASET / 'val' / 'labels' / (section2_image.stem + '.txt'),
    class_mapping
)

print("\nSection 2 dataset created!")


In [ ]:
# Create dataset config for Section 2
section2_config = {
    'path': str(SECTION2_DATASET.absolute()),
    'train': 'train/images',
    'val': 'val/images',
    'nc': len(all_classes_combined),
    'names': all_classes_combined
}

section2_config_path = SECTION2_DATASET / 'dataset.yaml'
with open(section2_config_path, 'w') as f:
    yaml.dump(section2_config, f, default_flow_style=False)

print(f"Section 2 dataset config saved to: {section2_config_path}")

# Strategy: Fine-tune the model with new classes
# We'll use transfer learning - start from pre-trained weights but allow the model
# to learn new classes

print("\n" + "="*60)
print("FINE-TUNING STRATEGY")
print("="*60)
print("1. Load pre-trained model from Section 1")
print("2. Extend model to include new classes")
print("3. Fine-tune on Section 2 data with low learning rate")
print("4. This allows the model to detect both old and new logos")
print("="*60)


In [ ]:
# Fine-tune the model with new classes
# We need to modify the model to support the new number of classes
print("Fine-tuning model with new classes...")

# Create a new model with extended classes
# YOLO will automatically handle class extension if we provide a model with different num_classes
# We'll fine-tune with a very low learning rate to preserve existing knowledge

finetuned_model = YOLO(pretrained_model_path)

# Fine-tune with new classes
# Use a very low learning rate to preserve pre-trained knowledge
finetune_results = finetuned_model.train(
    data=str(section2_config_path),
    epochs=50,
    imgsz=640,
    batch=4,  # Small batch for small dataset
    device=str(device),
    project='runs/detect',
    name='section2_finetuned',
    exist_ok=True,
    patience=10,
    save=True,
    plots=True,
    verbose=True,
    # Low learning rate for fine-tuning
    lr0=0.001,  # 10x lower than normal training
    lrf=0.01,
    # Data augmentation (but be careful with small dataset)
    hsv_h=0.01,
    hsv_s=0.5,
    hsv_v=0.3,
    degrees=5,
    translate=0.05,
    scale=0.3,
    flipud=0.0,
    fliplr=0.5,
    mosaic=0.5,  # Reduced mosaic for small dataset
    mixup=0.0,  # No mixup for very small dataset
    copy_paste=0.0,  # No copy-paste for very small dataset
)

print("\nFine-tuning completed!")
print(f"Results saved to: {finetune_results.save_dir}")


In [ ]:
# Load the fine-tuned model
finetuned_best = YOLO(finetune_results.save_dir / 'weights' / 'best.pt')

# Test on the Section 2 image
print("Testing fine-tuned model on Section 2 image...")
predictions_finetuned = finetuned_best.predict(
    source=str(section2_image),
    conf=0.25,
    iou=0.45,
    save=False,
    show=False
)

pred_ft = predictions_finetuned[0]

# Visualize fine-tuned predictions
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Ground truth
axes[0].imshow(img)
axes[0].axis('off')
axes[0].set_title('Ground Truth', fontsize=14, fontweight='bold')

img_width, img_height = img.size
for ann in annotations:
    parts = ann.strip().split()
    if len(parts) >= 5:
        class_name = parts[0]
        x_center, y_center, width, height = [float(x) for x in parts[1:5]]
        x = (x_center - width/2) * img_width
        y = (y_center - height/2) * img_height
        w = width * img_width
        h = height * img_height
        from matplotlib.patches import Rectangle
        rect = Rectangle((x, y), w, h, linewidth=3, edgecolor='red', facecolor='none')
        axes[0].add_patch(rect)
        axes[0].text(x, y-10, class_name, color='red', fontsize=10, fontweight='bold',
                    bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.8))

# Fine-tuned predictions
axes[1].imshow(img)
axes[1].axis('off')
axes[1].set_title('Fine-tuned Model Predictions', fontsize=14, fontweight='bold')

if pred_ft.boxes is not None and len(pred_ft.boxes) > 0:
    boxes = pred_ft.boxes.xyxy.cpu().numpy()
    confidences = pred_ft.boxes.conf.cpu().numpy()
    classes = pred_ft.boxes.cls.cpu().numpy().astype(int)
    
    print(f"\nFine-tuned model detected {len(boxes)} objects:")
    for box, conf, cls in zip(boxes, confidences, classes):
        x1, y1, x2, y2 = box
        if cls < len(all_classes_combined):
            class_name = all_classes_combined[cls]
            print(f"  {class_name}: {conf:.3f}")
            
            from matplotlib.patches import Rectangle
            rect = Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2, 
                           edgecolor='cyan', facecolor='none')
            axes[1].add_patch(rect)
            axes[1].text(x1, y1-5, f"{class_name} {conf:.2f}", 
                       color='cyan', fontsize=10, fontweight='bold',
                       bbox=dict(boxstyle='round,pad=0.3', facecolor='black', alpha=0.7))
else:
    print("\nNo objects detected by fine-tuned model")

plt.tight_layout()
plt.savefig('section2_finetuned_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nComparison saved to section2_finetuned_comparison.png")


## 5. Alternative Approach: Template Matching + Object Detection

For scenarios with very limited data, we can use a hybrid approach:
1. Extract logo templates from the provided image
2. Use template matching for detection
3. Combine with object detection for better accuracy


In [ ]:
# Extract logo templates from the annotated image
def extract_logo_templates(image_path, label_path, output_dir):
    """Extract logo regions as templates for template matching"""
    img = cv2.imread(str(image_path))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]
    
    templates = []
    
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 5:
                class_name = parts[0]
                x_center, y_center, width, height = [float(x) for x in parts[1:5]]
                
                # Convert to pixel coordinates
                x1 = int((x_center - width/2) * w)
                y1 = int((y_center - height/2) * h)
                x2 = int((x_center + width/2) * w)
                y2 = int((y_center + height/2) * h)
                
                # Extract template with padding
                padding = 5
                x1 = max(0, x1 - padding)
                y1 = max(0, y1 - padding)
                x2 = min(w, x2 + padding)
                y2 = min(h, y2 + padding)
                
                template = img_rgb[y1:y2, x1:x2]
                templates.append({
                    'class': class_name,
                    'template': template,
                    'bbox': (x1, y1, x2, y2)
                })
    
    return templates

# Extract templates
templates = extract_logo_templates(section2_image, section2_label, None)

print(f"Extracted {len(templates)} logo templates")

# Visualize extracted templates
fig, axes = plt.subplots(1, len(templates), figsize=(4*len(templates), 4))
if len(templates) == 1:
    axes = [axes]

for idx, template_info in enumerate(templates):
    axes[idx].imshow(template_info['template'])
    axes[idx].axis('off')
    axes[idx].set_title(f"{template_info['class']}\n{template_info['template'].shape[:2]}", 
                       fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig('section2_templates.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nTemplates saved to section2_templates.png")


In [ ]:
# Template matching function
def template_matching(image, templates, threshold=0.6):
    """Perform template matching on image"""
    img_gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
    detections = []
    
    for template_info in templates:
        template = template_info['template']
        template_gray = cv2.cvtColor(template, cv2.COLOR_RGB2GRAY)
        
        # Multi-scale template matching
        scales = [0.5, 0.75, 1.0, 1.25, 1.5]
        best_match_val = 0
        best_match_loc = None
        best_scale = 1.0
        
        for scale in scales:
            if scale != 1.0:
                h, w = template_gray.shape[:2]
                scaled_template = cv2.resize(template_gray, (int(w*scale), int(h*scale)))
            else:
                scaled_template = template_gray
            
            if scaled_template.shape[0] > img_gray.shape[0] or scaled_template.shape[1] > img_gray.shape[1]:
                continue
                
            result = cv2.matchTemplate(img_gray, scaled_template, cv2.TM_CCOEFF_NORMED)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
            
            if max_val > best_match_val:
                best_match_val = max_val
                best_match_loc = max_loc
                best_scale = scale
        
        if best_match_val >= threshold and best_match_loc is not None:
            h, w = template_gray.shape[:2]
            x1 = best_match_loc[0]
            y1 = best_match_loc[1]
            x2 = x1 + int(w * best_scale)
            y2 = y1 + int(h * best_scale)
            
            detections.append({
                'class': template_info['class'],
                'bbox': (x1, y1, x2, y2),
                'confidence': float(best_match_val)
            })
    
    return detections

# Test template matching
img_array = np.array(img)
template_detections = template_matching(img, templates, threshold=0.5)

print(f"Template matching found {len(template_detections)} detections:")
for det in template_detections:
    print(f"  {det['class']}: {det['confidence']:.3f} at {det['bbox']}")

# Visualize template matching results
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.imshow(img)
ax.axis('off')
ax.set_title('Template Matching Results', fontsize=14, fontweight='bold')

for det in template_detections:
    x1, y1, x2, y2 = det['bbox']
    from matplotlib.patches import Rectangle
    rect = Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2, 
                    edgecolor='magenta', facecolor='none')
    ax.add_patch(rect)
    ax.text(x1, y1-5, f"{det['class']} {det['confidence']:.2f}", 
           color='magenta', fontsize=10, fontweight='bold',
           bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.savefig('section2_template_matching.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nTemplate matching results saved to section2_template_matching.png")


## 6. Summary and Recommendations


In [ ]:
# Save Section 2 results
section2_results = {
    'new_classes': new_classes,
    'new_only_classes': new_only,
    'all_classes_combined': all_classes_combined,
    'finetuned_model_path': str(finetune_results.save_dir / 'weights' / 'best.pt'),
    'template_matching_detections': len(template_detections),
    'finetuned_detections': len(pred_ft.boxes) if pred_ft.boxes is not None else 0
}

with open('section2_results.json', 'w') as f:
    json.dump(section2_results, f, indent=2)

print("="*60)
print("SECTION 2 SUMMARY")
print("="*60)
print(f"New classes detected: {new_classes}")
print(f"Truly new classes (not in original): {new_only}")
print(f"\nApproaches tested:")
print(f"  1. Pre-trained model: May detect similar logos")
print(f"  2. Fine-tuning: Extends model to new classes")
print(f"  3. Template matching: Works for exact logo matches")
print(f"\nRecommendations:")
print(f"  - For production: Use fine-tuned model with more training data")
print(f"  - For quick prototyping: Use template matching")
print(f"  - Hybrid approach: Combine both methods")
print("="*60)

print("\nSection 2 results saved to section2_results.json")
